In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, confusion_matrix
from sklearn.preprocessing import label_binarize

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
# from OpenMax import openmax



import time

In [2]:
head = {
            "model" : '',
            "attack_model": '',
            'epsilon': '',
            'Accuracy': '',
            'Macro Precision': '',
            'Weighted Precision': '',
            'Macro Recall': '',
            'Weighted Recall': '',
            'Macro F1': '',
            'Weighted F1': '',

        }
head = pd.DataFrame([head])
head.to_csv("./RGAN.csv", mode='a', index=False)


In [3]:
# def calculate_performance_metrics(X_test, y_test, model, model_name, attack_name, eps):
#     model.eval()
#     device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#     model.to(device)
    
#     all_preds = []
#     all_labels = []
#     probabilities = []

#     num_classes = len(np.unique(y_test))
    
#     X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
#     y_test_tensor = torch.tensor(y_test, dtype=torch.long)
#     test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
#     test_loader = DataLoader(dataset=test_dataset)

#     with torch.no_grad():
        
#         for inputs, labels in test_loader:
#             inputs, labels = inputs.to(device), labels.to(device)
#             outputs = model(inputs)
#             preds = torch.argmax(outputs, dim=1)
#             all_preds.extend(preds.cpu().numpy())
#             all_labels.extend(labels.cpu().numpy())
#             probabilities.extend(torch.nn.functional.softmax(outputs, dim=1).cpu().numpy())
        
#         all_preds = np.array(all_preds)
#         all_labels = np.array(all_labels)
#         probabilities = np.array(probabilities)

#         np.save(f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Defense_Label/UNSW_Def11/y_pred_{attack_name}{eps}_Def11.npy", all_preds)

        

In [4]:
def calculate_performance_metrics(X_test, y_test, model, model_name, attack_name, eps):
    model.eval()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    all_preds = []
    all_labels = []
    probabilities = []

    num_classes = len(np.unique(y_test))
    
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
    test_loader = DataLoader(dataset=test_dataset)

    with torch.no_grad():
        
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            probabilities.extend(torch.nn.functional.softmax(outputs, dim=1).cpu().numpy())
        
        all_preds = np.array(all_preds)
        all_labels = np.array(all_labels)
        probabilities = np.array(probabilities)
        
        accuracy = accuracy_score(all_labels, all_preds)

        precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(all_labels, all_preds, average='macro')
        precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')
    

        print(f"Accuracy: {accuracy}")
        
        print("\nmacro")
        print(f"Precision: {precision_macro}\nRecall: {recall_macro}\nF1 Score: {f1_macro}")
    
        print("\nweighted")
        print(f"Precision: {precision_weighted}\nRecall: {recall_weighted}\nF1 Score: {f1_weighted}")
        print()
        


        new_row = {
            "model" : model_name,
            "attack_model" : attack_name,
            'epsilon': eps,
            'Accuracy': accuracy,
            'Macro Precision': precision_macro,
            'Weighted Precision': precision_weighted,
            'Macro Recall': recall_macro,
            'Weighted Recall': recall_weighted,
            'Macro F1': f1_macro,
            'Weighted F1': f1_weighted,

        }
        new_row_df = pd.DataFrame([new_row])
        new_row_df.to_csv("./RGAN.csv", mode='a', index=False, header=False)



In [5]:
x_test = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
x_train = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_train.npy')
x_val = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_val.npy')
y_test = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')
y_train = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_train.npy')
y_val = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_val.npy')

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

Using cuda device


In [7]:
x_train_tensor = torch.tensor(x_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)

x_val_tensor = torch.tensor(x_val, dtype=torch.float32).to(device)
y_val_tensor = torch.tensor(y_val, dtype=torch.long).to(device)

train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True)

val_dataset = TensorDataset(x_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=100, shuffle=True)

In [8]:
# class DNNModel(nn.Module):
#     def __init__(self, input_size, output_size):
#         super(DNNModel, self).__init__()
#         self.fc1 = nn.Linear(input_size, 50)
#         self.bn1 = nn.BatchNorm1d(50)
#         self.fc2 = nn.Linear(50, 30)
#         self.bn2 = nn.BatchNorm1d(30)
#         self.fc3 = nn.Linear(30, 20)
#         self.bn3 = nn.BatchNorm1d(20)
#         self.fc4 = nn.Linear(20, output_size)

#     def forward(self, x):
#         x = self.bn1(F.relu(self.fc1(x)))
#         x = self.bn2(F.relu(self.fc2(x)))
#         x = self.bn3(F.relu(self.fc3(x)))
#         logits = self.fc4(x)
#         return logits

class DNNModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(DNNModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, 30)
        self.fc3 = nn.Linear(30, 20)
        self.fc4 = nn.Linear(20, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

class Generator(nn.Module):
    def __init__(self, input_size):
        super(Generator, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, input_size)

    def forward(self, x):
        perturbation = torch.tanh(self.fc1(x))
        perturbation = self.fc2(perturbation)
        perturbation = torch.clamp(perturbation, -0.1, 0.1)
        return perturbation

class Discriminator(nn.Module):
    def __init__(self, input_size):
        super(Discriminator, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x

In [9]:
input_size = x_train.shape[1]
output_size = len(np.unique(y_train))

In [10]:
def train_rgan(model, G, D, train_loader, val_loader, device, epochs=15, lr=0.001, patience=3, min_delta=0.001):
    optimizer_G = optim.Adam(G.parameters(), lr=lr)
    optimizer_D = optim.Adam(D.parameters(), lr=lr)
    optimizer_M = optim.Adam(model.parameters(), lr=lr)
    loss_function = nn.CrossEntropyLoss()

    best_loss = float("inf")
    patience_counter = 0

    for epoch in range(epochs):
        model.train()
        G.train()
        D.train()

        train_loss = 0.0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            adv_inputs = inputs + G(inputs).detach().to(device)
            adv_inputs = torch.clamp(adv_inputs, -1, 1)

            optimizer_D.zero_grad()
            real_loss = -torch.mean(D(inputs))
            fake_loss = torch.mean(D(G(inputs)))
            loss_D = real_loss + fake_loss
            loss_D.backward()
            optimizer_D.step()

            optimizer_G.zero_grad()
            fake_loss = -torch.mean(D(G(inputs)))
            fake_loss.backward()
            optimizer_G.step()

            optimizer_M.zero_grad()
            outputs = model(adv_inputs)
            loss_M = loss_function(outputs, labels)
            loss_M.backward()
            optimizer_M.step()

            train_loss += loss_M.item()

        avg_train_loss = train_loss / len(train_loader)

        model.eval()
        val_loss = 0.0
        correct_predictions = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = loss_function(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                correct_predictions += (predicted == labels).sum().item()

        avg_val_loss = val_loss / len(val_loader)
        val_accuracy = correct_predictions / len(val_loader.dataset)

        print(f"Epoch {epoch+1}, Training Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

        if best_loss - avg_val_loss > min_delta:
            best_loss = avg_val_loss
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= patience:
            print("Early stopping triggered")
            break


In [11]:
model = DNNModel(input_size, output_size).to(device)
G = Generator(input_size).to(device)
D = Discriminator(input_size).to(device)



In [12]:
# train_rgan(model, G, D, train_loader, val_loader, device)


In [13]:
# torch.save(model.state_dict(), "./RGAN.pt")

In [14]:
model.load_state_dict(torch.load("/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Defense/RGAN/RGAN.pt"))

/tmp/ipykernel_183/2756394764.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Defense/RGA

<All keys matched successfully>

In [15]:
# calculate_performance_metrics(x_test, y_test, model, 'DNN', 'baseline', '0')

In [16]:
def print_empty_file():
    new_row = {
        "model" : "0",
        "attack_model" : "0",
        'epsilon': "0",
        'Accuracy': "0",
        'Macro Precision': "0",
        'Weighted Precision': "0",
        'Macro Recall': "0",
        'Weighted Recall': "0",
        'Macro F1': "0",
        'Weighted F1': "0",
    }
    new_row_df = pd.DataFrame([new_row])
    new_row_df.to_csv("./RGAN.csv", mode='a', index=False, header=False)

In [13]:
epsilon = 0
Def = "Def11"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

percentage = ["100", "50", "20", "1"]
model_name = ["XGB", "RF", "DT" ]

base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

for m_name in model_name:
    for p in percentage:
        for attack in attack_names:
            print(f"start {m_name} {p} {attack}")
            
            x_path = f"{base_path}/{m_name}/UNSW_Input{p}/x_test_adv_{attack}_{Def}.npy"
            y_path = f"{base_path}/{m_name}/UNSW_Input{p}/y_test_adv_{attack}_{Def}.npy"

            try:
                x_test_adv = np.load(x_path)
                y_test_adv = np.load(y_path)
                print(x_test_adv.shape, y_test_adv.shape)

                m_per_name = f"{m_name}{p}"
                calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
            except FileNotFoundError:
                print(x_path, "not found")
                print_empty_file()


start XGB 100 baseline
(64, 56) (64,)
Accuracy: 0.8125

macro
Precision: 0.4406779661016949
Recall: 0.45614035087719296
F1 Score: 0.4482758620689655

weighted
Precision: 0.784957627118644
Recall: 0.8125
F1 Score: 0.7984913793103449

start XGB 100 BIM
(100, 56) (100,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.p

Accuracy: 0.15

macro
Precision: 0.5
Recall: 0.075
F1 Score: 0.13043478260869565

weighted
Precision: 1.0
Recall: 0.15
F1 Score: 0.2608695652173913

start XGB 100 FGSM
(52, 56) (52,)
Accuracy: 0.038461538461538464

macro
Precision: 0.5
Recall: 0.019230769230769232
F1 Score: 0.037037037037037035

weighted
Precision: 1.0
Recall: 0.038461538461538464
F1 Score: 0.07407407407407407

start XGB 100 PGD
(100, 56) (100,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.15

macro
Precision: 0.5
Recall: 0.075
F1 Score: 0.13043478260869565

weighted
Precision: 1.0
Recall: 0.15
F1 Score: 0.2608695652173913

start XGB 100 DF
(132, 56) (132,)
Accuracy: 0.12878787878787878

macro
Precision: 0.08018867924528301
Recall: 0.19767441860465115
F1 Score: 0.11409395973154363

weighted
Precision: 0.052244139508290446
Recall: 0.12878787878787878
F1 Score: 0.07433394346146024

start XGB 100 AutoPGD
(188, 56) (188,)


...

Accuracy: 0.5376344086021505

macro
Precision: 0.36764705882352944
Recall: 0.3333333333333333
F1 Score: 0.34965034965034963

weighted
Precision: 0.5929791271347249
Recall: 0.5376344086021505
F1 Score: 0.5639521768554027

start XGB 1 SINIFGSM
(15, 56) (15,)
Accuracy: 0.13333333333333333

macro
Precision: 0.125
Recall: 0.1111111111111111
F1 Score: 0.11764705882352941

weighted
Precision: 0.15
Recall: 0.13333333333333333
F1 Score: 0.1411764705882353

start XGB 1 VNIFGSM
(7, 56) (7,)
Accuracy: 0.2857142857142857

macro
Precision: 0.16666666666666666
Recall: 0.3333333333333333
F1 Score: 0.2222222222222222

weighted
Precision: 0.14285714285714285
Recall: 0.2857142857142857
F1 Score: 0.19047619047619047

start RF 100 baseline
(108, 56) (108,)
Accuracy: 0.75

macro
Precision: 0.5246913580246914
Recall: 0.6298076923076923
F1 Score: 0.4915431560592851

weighted
Precision: 0.9419295839048926
Recall: 0.75
F1 Score: 0.8272013949433306

start RF 100 BIM
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.24352331606217617

macro
Precision: 0.5
Recall: 0.12176165803108809
F1 Score: 0.19583333333333333

weighted
Precision: 1.0
Recall: 0.24352331606217617
F1 Score: 0.39166666666666666

start RF 100 SINIFGSM
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF/UNSW_Input100/x_test_adv_SINIFGSM_Def11.npy not found
start RF 100 VNIFGSM
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF/UNSW_Input100/x_test_adv_VNIFGSM_Def11.npy not found
start RF 50 baseline
(80, 56) (80,)
Accuracy: 0.775

macro
Precision: 0.5591397849462366
Recall: 0.6486486486486487
F1 Score: 0.5588235294117647

weighted
Precision: 0.892741935483871
Recall: 0.775
F1 Score: 0.8213235294117647

start RF 50 BIM
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF/UNSW_Input50/x_test_adv_BIM_Def11.npy not found
start RF 50 FGSM
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF/UNSW_Input50/x_test_adv_FGSM_Def11.npy not found
start 

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/m

Accuracy: 0.0

macro
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

weighted
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

start RF 1 AutoPGD
(9, 56) (9,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start RF 1 ZOO
(15, 56) (15,)
Accuracy: 0.6

macro
Precision: 0.3
Recall: 0.5
F1 Score: 0.375

weighted
Precision: 0.36
Recall: 0.6
F1 Score: 0.45

start RF 1 CaFA
(11, 56) (11,)
Accuracy: 0.18181818181818182

macro
Precision: 0.125
Recall: 0.2
F1 Score: 0.15384615384615385

weighted
Precision: 0.11363636363636363
Recall: 0.18181818181818182
F1 Score: 0.13986013986013987

start RF 1 SINIFGSM
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF/UNSW_Input1/x_test_adv_SINIFGSM_Def11.npy not found
start RF 1 VNIFGSM
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF/UNSW_Input1/x_test_adv_VNIFGSM_Def11.npy not found
start DT 100 baseline
(75, 56) (75,)
Accuracy: 0.8

macro
Precision: 0.476190

In [15]:
epsilon = 0
Def = "Def11"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

percentage = ["100", "50", "20", "1"]
model_name = ["XGB", "RF", "DT" ]

base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

for m_name in model_name:
    for p in percentage:
        for attack in attack_names:
            print(f"start {m_name} {p} {attack}")
            
            x_path = f"{base_path}/{m_name}_ExcludeCaFA/UNSW_Input{p}/x_test_adv_{attack}_{Def}.npy"
            y_path = f"{base_path}/{m_name}_ExcludeCaFA/UNSW_Input{p}/y_test_adv_{attack}_{Def}.npy"

            try:
                x_test_adv = np.load(x_path)
                y_test_adv = np.load(y_path)
                print(x_test_adv.shape, y_test_adv.shape)

                m_per_name = f"{m_name}{p}_ExcludeCaFA"
                calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
            except FileNotFoundError:
                print(x_path, "not found")
                print_empty_file()


start XGB 100 baseline
(84, 56) (84,)
Accuracy: 0.8571428571428571

macro
Precision: 0.47368421052631576
Recall: 0.45
F1 Score: 0.46153846153846156

weighted
Precision: 0.9022556390977443
Recall: 0.8571428571428571
F1 Score: 0.8791208791208792

start XGB 100 BIM
(25, 56) (25,)
Accuracy: 0.24

macro
Precision: 0.5
Recall: 0.12
F1 Score: 0.1935483870967742

weighted
Precision: 1.0
Recall: 0.24
F1 Score: 0.3870967741935484

start XGB 100 FGSM
(49, 56) (49,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.12244897959183673

macro
Precision: 0.3333333333333333
Recall: 0.06521739130434782
F1 Score: 0.10909090909090909

weighted
Precision: 0.6258503401360543
Recall: 0.12244897959183673
F1 Score: 0.2048237476808905

start XGB 100 PGD
(25, 56) (25,)
Accuracy: 0.24

macro
Precision: 0.5
Recall: 0.12
F1 Score: 0.1935483870967742

weighted
Precision: 1.0
Recall: 0.24
F1 Score: 0.3870967741935484

start XGB 100 DF
(152, 56) (152,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.09868421052631579

macro
Precision: 0.06944444444444445
Recall: 0.1271186440677966
F1 Score: 0.08982035928143713

weighted
Precision: 0.05391081871345029
Recall: 0.09868421052631579
F1 Score: 0.06972896312637883

start XGB 100 AutoPGD
(240, 56) (240,)


...

Accuracy: 0.2727272727272727

macro
Precision: 0.5
Recall: 0.13636363636363635
F1 Score: 0.21428571428571427

weighted
Precision: 1.0
Recall: 0.2727272727272727
F1 Score: 0.42857142857142855

start DT 50 SINIFGSM
(140, 56) (140,)
Accuracy: 0.65

macro
Precision: 0.6029411764705883
Recall: 0.5894819466248038
F1 Score: 0.5914477994163541

weighted
Precision: 0.6338235294117647
Recall: 0.65
F1 Score: 0.6378476564826395

start DT 50 VNIFGSM
(145, 56) (145,)
Accuracy: 0.45517241379310347

macro
Precision: 0.460519342872284
Recall: 0.47154316271963337
F1 Score: 0.42347375308269164

weighted
Precision: 0.46114430860881167
Recall: 0.45517241379310347
F1 Score: 0.41508293112993555

start DT 20 baseline
(205, 56) (205,)
Accuracy: 0.5219512195121951

macro
Precision: 0.5854642502482621
Recall: 0.5677390791027155
F1 Score: 0.5107636859536333

weighted
Precision: 0.6092014435536609
Recall: 0.5219512195121951
F1 Score: 0.49741082331922093

start DT 20 BIM
(347, 56) (347,)
Accuracy: 0.195965417867435

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.29577464788732394

macro
Precision: 0.5
Recall: 0.14788732394366197
F1 Score: 0.22826086956521738

weighted
Precision: 1.0
Recall: 0.29577464788732394
F1 Score: 0.45652173913043476

start DT 20 SINIFGSM
(1029, 56) (1029,)
Accuracy: 0.8765792031098154

macro
Precision: 0.5013056587519518
Recall: 0.5237047898338221
F1 Score: 0.4748333273053878

weighted
Precision: 0.9887299175156136
Recall: 0.8765792031098154
F1 Score: 0.928806166964391

start DT 20 VNIFGSM
(206, 56) (206,)
Accuracy: 0.6553398058252428

macro
Precision: 0.494007914075749
Recall: 0.4869071146245059
F1 Score: 0.46438642106419603

weighted
Precision: 0.805147991021201
Recall: 0.6553398058252428
F1 Score: 0.7158860009933795

start DT 1 baseline
(88, 56) (88,)
Accuracy: 0.6477272727272727

macro
Precision: 0.6477272727272727
Recall: 0.6541778975741239
F1 Score: 0.6440036539214407

weighted
Precision: 0.6686466942148761
Recall: 0.6477272727272727
F1 Score: 0.6514508915331048

start DT 1 BIM
(161, 56) (161,)
Accurac

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.7708333333333334

macro
Precision: 0.5
Recall: 0.3854166666666667
F1 Score: 0.43529411764705883

weighted
Precision: 1.0
Recall: 0.7708333333333334
F1 Score: 0.8705882352941177

start DT 1 SINIFGSM
(281, 56) (281,)
Accuracy: 0.17437722419928825

macro
Precision: 0.398482498892335
Recall: 0.3908668730650155
F1 Score: 0.17428310872428818

weighted
Precision: 0.6290835786489483
Recall: 0.17437722419928825
F1 Score: 0.18096530744929393

start DT 1 VNIFGSM
(86, 56) (86,)
Accuracy: 0.3372093023255814

macro
Precision: 0.36447368421052634
Recall: 0.43958944281524925
F1 Score: 0.2951833213515457

weighted
Precision: 0.3743880048959608
Recall: 0.3372093023255814
F1 Score: 0.24715362880979053



In [13]:
epsilon = 0
Def = "Def11"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

percentage = ["100", "50", "20", "1"]
model_name = ["XGB", "RF", "DT" ]

base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

total_num_attack = ["2", "3"]
for num_attack in total_num_attack:
    for m_name in model_name:
        for p in percentage:
            for attack in attack_names:
                print(f"start {m_name} {p} {attack}")
                
                x_path = f"{base_path}/{m_name}_Exclude{num_attack}Attack/UNSW_Input{p}/x_test_adv_{attack}_{Def}.npy"
                y_path = f"{base_path}/{m_name}_Exclude{num_attack}Attack/UNSW_Input{p}/y_test_adv_{attack}_{Def}.npy"
    
                try:
                    x_test_adv = np.load(x_path)
                    y_test_adv = np.load(y_path)
                    print(x_test_adv.shape, y_test_adv.shape)
    
                    m_per_name = f"{m_name}{p}_Exclude{num_attack}Attack"
                    calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
                except FileNotFoundError:
                    print(x_path, "not found")
                    print_empty_file()


start XGB 100 baseline
(94, 56) (94,)
Accuracy: 0.8617021276595744

macro
Precision: 0.7428571428571429
Recall: 0.8874643874643875
F1 Score: 0.7812779667084303

weighted
Precision: 0.9185410334346504
Recall: 0.8617021276595744
F1 Score: 0.8772225797729531

start XGB 100 BIM
(33, 56) (33,)
Accuracy: 0.36363636363636365

macro
Precision: 0.5
Recall: 0.18181818181818182
F1 Score: 0.26666666666666666

weighted
Precision: 1.0
Recall: 0.36363636363636365
F1 Score: 0.5333333333333333

start XGB 100 FGSM
(25, 56) (25,)
Accuracy: 0.16

macro
Precision: 0.5
Recall: 0.08
F1 Score: 0.13793103448275862

weighted
Precision: 1.0
Recall: 0.16
F1 Score: 0.27586206896551724

start XGB 100 PGD
(33, 56) (33,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.p

Accuracy: 0.36363636363636365

macro
Precision: 0.5
Recall: 0.18181818181818182
F1 Score: 0.26666666666666666

weighted
Precision: 1.0
Recall: 0.36363636363636365
F1 Score: 0.5333333333333333

start XGB 100 DF
(140, 56) (140,)
Accuracy: 0.1357142857142857

macro
Precision: 0.09817813765182186
Recall: 0.2144569647566531
F1 Score: 0.12277976282947543

weighted
Precision: 0.07514459224985541
Recall: 0.1357142857142857
F1 Score: 0.08169363131301924

start XGB 100 AutoPGD
(245, 56) (245,)
Accuracy: 0.8530612244897959

macro
Precision: 0.5263157894736842
Recall: 0.9259259259259259
F1 Score: 0.51

weighted
Precision: 0.9922663802363051
Recall: 0.8530612244897959
F1 Score: 0.9133061224489796

start XGB 100 ZOO
(247, 56) (247,)
Accuracy: 0.8340080971659919

macro
Precision: 0.49282296650717705
Recall: 0.42213114754098363
F1 Score: 0.45474613686534215

weighted
Precision: 0.9736745249210623
Recall: 0.8340080971659919
F1 Score: 0.8984458088675585

start XGB 100 CaFA
(18, 56) (18,)
Accuracy: 0.722

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.p

Accuracy: 0.8524590163934426

macro
Precision: 0.7
Recall: 0.9181818181818182
F1 Score: 0.7411598302687412

weighted
Precision: 0.940983606557377
Recall: 0.8524590163934426
F1 Score: 0.8775013332715005

start XGB 50 BIM
(24, 56) (24,)
Accuracy: 0.6666666666666666

macro
Precision: 0.47058823529411764
Recall: 0.34782608695652173
F1 Score: 0.4

weighted
Precision: 0.9019607843137255
Recall: 0.6666666666666666
F1 Score: 0.7666666666666667

start XGB 50 FGSM
(35, 56) (35,)
Accuracy: 0.4

macro
Precision: 0.5
Recall: 0.2
F1 Score: 0.2857142857142857

weighted
Precision: 1.0
Recall: 0.4
F1 Score: 0.5714285714285714

start XGB 50 PGD
(24, 56) (24,)


...

Accuracy: 0.6588868940754039

macro
Precision: 0.5451169072338989
Recall: 0.6221249047981721
F1 Score: 0.5100462962962963

weighted
Precision: 0.86522749132832
Recall: 0.6588868940754039
F1 Score: 0.7296710220094421

start DT 100 VNIFGSM
(6462, 56) (6462,)
Accuracy: 0.9798823893531414

macro
Precision: 0.8476619743327857
Recall: 0.5697896238188626
F1 Score: 0.6125506184910847

weighted
Precision: 0.9751764488872281
Recall: 0.9798823893531414
F1 Score: 0.9732269821887621

start DT 50 baseline
(200, 56) (200,)
Accuracy: 0.63

macro
Precision: 0.6261115179412072
Recall: 0.6215344288738784
F1 Score: 0.6214833759590792

weighted
Precision: 0.6277780102521184
Recall: 0.63
F1 Score: 0.6265933503836316

start DT 50 BIM
(913, 56) (913,)
Accuracy: 0.5323110624315444

macro
Precision: 0.5764217076244487
Recall: 0.6037797654860749
F1 Score: 0.5155138323956807

weighted
Precision: 0.7239642105110596
Recall: 0.5323110624315444
F1 Score: 0.5634353416156447

start DT 50 FGSM
(942, 56) (942,)
Accuracy:

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.4452054794520548

macro
Precision: 0.5
Recall: 0.2226027397260274
F1 Score: 0.3080568720379147

weighted
Precision: 1.0
Recall: 0.4452054794520548
F1 Score: 0.6161137440758294

start DT 20 SINIFGSM
(119, 56) (119,)
Accuracy: 0.5378151260504201

macro
Precision: 0.5161392405063291
Recall: 0.5147058823529412
F1 Score: 0.510727367870225

weighted
Precision: 0.5255877034358047
Recall: 0.5378151260504201
F1 Score: 0.5271735067653435

start DT 20 VNIFGSM
(433, 56) (433,)
Accuracy: 0.30484988452655887

macro
Precision: 0.4787325078022752
Recall: 0.3976997578692494
F1 Score: 0.2550058589842522

weighted
Precision: 0.8830867737629647
Recall: 0.30484988452655887
F1 Score: 0.4299049843246677

start DT 1 baseline
(80, 56) (80,)
Accuracy: 0.625

macro
Precision: 0.6063492063492063
Recall: 0.6170510132774283
F1 Score: 0.6050032916392363

weighted
Precision: 0.6548412698412698
Recall: 0.625
F1 Score: 0.6338874259381172

start DT 1 BIM
(114, 56) (114,)
Accuracy: 0.2982456140350877

macro
P

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.6415094339622641

macro
Precision: 0.5
Recall: 0.32075471698113206
F1 Score: 0.39080459770114945

weighted
Precision: 1.0
Recall: 0.6415094339622641
F1 Score: 0.7816091954022989

start DT 1 SINIFGSM
(87, 56) (87,)
Accuracy: 0.25287356321839083

macro
Precision: 0.24903225806451612
Recall: 0.28626373626373625
F1 Score: 0.24287053153032534

weighted
Precision: 0.24726733407489804
Recall: 0.25287356321839083
F1 Score: 0.22586537766061404

start DT 1 VNIFGSM
(136, 56) (136,)
Accuracy: 0.20588235294117646

macro
Precision: 0.3652735304111451
Recall: 0.3358178053830228
F1 Score: 0.20433369447453953

weighted
Precision: 0.5479902460474505
Recall: 0.20588235294117646
F1 Score: 0.22859601045185138



In [17]:
epsilon = 0

Def = "Def11"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

rec_list = ["Euclidean", "cosine", "manhattan"]
for rec in rec_list:
    for attack in attack_names:
        print(f"start {attack}")
        
        x_path = f"{base_path}/Recommendation_{rec}/x_test_adv_{attack}_{Def}.npy"
        y_path = f"{base_path}/Recommendation_{rec}/y_test_adv_{attack}_{Def}.npy"
    
        try:
            x_test_adv = np.load(x_path)
            y_test_adv = np.load(y_path)
            m_per_name = f"Recommendation_{rec}"
            calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
        except FileNotFoundError:
            print(x_path, "not found")
            print_empty_file()



start baseline
Accuracy: 0.7647058823529411

macro
Precision: 0.6364535266974292
Recall: 0.8004354136429608
F1 Score: 0.645531914893617

weighted
Precision: 0.901508395052154
Recall: 0.7647058823529411
F1 Score: 0.8061576971214017

start BIM
Accuracy: 0.6640746500777605

macro
Precision: 0.5758720159827125
Recall: 0.7160291386115626
F1 Score: 0.5389212014978886

weighted
Precision: 0.8952263808096101
Recall: 0.6640746500777605
F1 Score: 0.735057203003658

start FGSM
Accuracy: 0.5641025641025641

macro
Precision: 0.5355481727574751
Recall: 0.5515911282545805
F1 Score: 0.5096153846153846

weighted
Precision: 0.6925973251554647
Recall: 0.5641025641025641
F1 Score: 0.6018244575936885

start PGD
Accuracy: 0.6640746500777605

macro
Precision: 0.5758720159827125
Recall: 0.7160291386115626
F1 Score: 0.5389212014978886

weighted
Precision: 0.8952263808096101
Recall: 0.6640746500777605
F1 Score: 0.735057203003658

start DF
Accuracy: 0.17857142857142858

macro
Precision: 0.20348837209302326
Recal